In [176]:
import torch
import torch.nn as nn
import torch.nn.functional
import torch_geometric.nn 
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import pandas as pd
import networkx as nx
import numpy as np
import tqdm
import random
from torch_geometric.nn import GATConv, global_mean_pool
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader


In [177]:
pkl_graphs = pd.read_pickle('/Users/MathildeStouby/Desktop/P5 GitHub/5-semester/Momentum graphs.pkl')

In [178]:
unique_nodes = []
for graph in pkl_graphs.values():
    temp = [node for node in graph.nodes() if node not in unique_nodes]
    unique_nodes.extend(temp)

In [179]:
idx_to_pos = dict(enumerate(unique_nodes))
pos_to_idx = {pos : idx for idx, pos in idx_to_pos.items()}

In [ ]:
pyg_data = []
thors_kamp = []


for idx, graph in pkl_graphs.items():

    closeness = nx.closeness_centrality(graph)
    betweenness = nx.closeness_centrality(graph)
    pagerank = nx.pagerank(graph, weight='weight')
    centrality_list = [closeness, betweenness, pagerank] 

    #adj_dict = nx.to_dict_of_dicts(graph)

    for node in list(graph.nodes()):
        #adj_vect = np.zeros((len(unique_nodes)))
        #players = adj_dict[node]
        #for key, value in players.items():
            #adj_vect[pos_to_idx[key]] = value['weight']
        #adj_vect = torch.from_numpy(adj_vect).float()
        centrality_vect = []
        for measure in centrality_list:
            if node in list(measure.keys()):
                centrality_vect.append(measure[node])
            else:
                centrality_vect.append(0)
        centrality_vect = torch.Tensor(centrality_vect).float()        
        #graph.nodes[node]['x'] = torch.cat((adj_vect, centrality_vect), -1)
        graph.nodes[node]['x'] = centrality_vect

   
    for node in unique_nodes:
        if node not in graph.nodes:
            graph.add_node(node) 
            #graph.nodes[node]['x'] = torch.from_numpy(np.zeros((len(unique_nodes)+3))).float()
            graph.nodes[node]['x'] = torch.from_numpy(np.zeros(3)).float()  
            

    data = from_networkx(graph)

    try:
        data['momentum']
        if idx.startswith('3895275'):
            thors_kamp.append(data)
        else:
            pyg_data.append(data)
    except:
        print(data)


Data(x=[23, 3], edge_index=[2, 22], weight=[22])


In [181]:
train_idx = random.sample(range(len(pyg_data)), int(len(pyg_data) * 0.8))
test_idx = [i for i in range(len(pyg_data)) if i not in train_idx]

In [182]:
train_data = [pyg_data[idx] for idx in train_idx]
dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [183]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1= GATConv(input_dim, hidden_dim, heads=num_heads, edge_dim=1)
        self.layer2= GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads, edge_dim=1)
        #self.layer3= GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads, edge_dim=1)
        #self.layer4= GATConv(hidden_dim * num_heads, hidden_dim, heads=num_heads)
        self.layer5 = GATConv(hidden_dim * num_heads, output_dim, heads=1, edge_dim=1, concat=False)
        self.activation_function = nn.ReLU()
     

    def forward(self, input, edge_index, edge_attr, batch):
        output = self.activation_function(self.layer1(input, edge_index, edge_attr))
        output = self.activation_function(self.layer2(output, edge_index, edge_attr))
        output = self.layer5(output, edge_index, edge_attr)
        output = global_mean_pool(output, batch)
        return output

In [191]:
#input_dim = len(unique_nodes)+3
input_dim = 3
lr = 0.01

gat = GAT(input_dim = input_dim, hidden_dim = 11, output_dim = 1, num_heads = 6)
optimizer = torch.optim.SGD(gat.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()
epochs_num = 100

In [192]:
for epoch in tqdm.tqdm(range(epochs_num)):
    
    epoch_loss = 0
    for batch in tqdm.tqdm(dataloader):
        # Forward pass
        optimizer.zero_grad()
        
        output = gat(batch.x, batch.edge_index, batch.weight, batch.batch)
        
        # Calculate loss
        loss = loss_fn(output, batch.momentum)
        loss.backward()

        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader):.4f}") 
gat.eval() 

  0%|          | 0/100 [00:00<?, ?it/s]/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([37])) that is different to the input size (torch.Size([37, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  1%|          | 1/100 [00:00<00:27,  3.57it/s]

Epoch 1, Loss: 0.0072


  2%|▏         | 2/100 [00:00<00:20,  4.71it/s]

Epoch 2, Loss: 0.0070


  3%|▎         | 3/100 [00:00<00:22,  4.35it/s]

...

 99%|█████████▉| 99/100 [00:18<00:00,  5.20it/s]

Epoch 99, Loss: 0.0069


100%|██████████| 100/100 [00:18<00:00,  5.32it/s]

Epoch 100, Loss: 0.0070


GAT(
  (layer1): GATConv(3, 11, heads=6)
  (layer2): GATConv(66, 11, heads=6)
  (layer5): GATConv(66, 1, heads=1)
  (activation_function): ReLU()
)

In [193]:
y_pred  = []
y_true = []

with torch.no_grad():
    for idx in test_idx:
        batch = torch.zeros(pyg_data[idx].x.size(0), dtype=torch.long)
        output = gat(pyg_data[idx].x, pyg_data[idx].edge_index, pyg_data[idx].weight, batch)
        y_pred.append(output.item())
        y_true.append(pyg_data[idx].momentum)
        
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

0.02662291302452502

In [194]:
y_pred

[0.054200295358896255,
 0.04919273033738136,
 0.04670429229736328,
 0.05131502076983452,
 0.04615010321140289,
 0.039218902587890625,
 0.03986019268631935,
 0.034883249551057816,
 0.04380013793706894,
 0.030633594840765,
 0.04836907982826233,
 0.050724051892757416,
 0.045164741575717926,
 0.051712699234485626,
 0.03040635772049427,
 0.048722196370363235,
 0.050163451582193375,
 0.03825452923774719,
 0.04538062959909439,
 0.04027505964040756,
 0.04897720739245415,
 0.05118934065103531,
 0.056384045630693436,
 0.04934357479214668,
 0.04433446377515793,
 0.05025928094983101,
 0.05447214096784592,
 0.055687833577394485,
 0.049616843461990356,
 0.045013755559921265,
 0.04644521325826645,
 0.042973630130290985,
 0.03463193029165268,
 0.047218624502420425,
 0.023461773991584778,
 0.04632022604346275,
 0.05409754440188408,
 0.0491645485162735,
 0.05118916928768158,
 0.05334385111927986,
 0.04013759270310402,
 0.05276736617088318,
 0.04702814668416977,
 0.048945557326078415,
 0.0396877527236938

In [195]:
y_pred  = []
y_true = []
batch = torch.zeros(23, dtype=torch.long)


with torch.no_grad():
    for graph in thors_kamp:
        output = gat(graph.x, graph.edge_index, graph.weight, batch)
        y_pred.append(output.item())
        y_true.append(graph.momentum)


In [196]:
for pair in zip(y_pred, y_true):
    print("Prediction = " + str(pair[0]) + ', Actual momementum = ' + str(pair[1].item()))

from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

Prediction = 0.05346691235899925, Actual momementum = 0.09109056740999222
Prediction = 0.05516462400555611, Actual momementum = -0.04241221025586128
Prediction = 0.03943176567554474, Actual momementum = 0.166485995054245
Prediction = 0.051137883216142654, Actual momementum = 0.06548559665679932
Prediction = 0.04667608067393303, Actual momementum = 0.08412574231624603
Prediction = 0.04482093080878258, Actual momementum = 0.103540800511837
Prediction = 0.04308966174721718, Actual momementum = 0.13816143572330475
Prediction = 0.0566927045583725, Actual momementum = -0.02740156091749668
Prediction = 0.05580773577094078, Actual momementum = 0.048808980733156204
Prediction = 0.03747383877635002, Actual momementum = 0.013777799904346466
Prediction = 0.04632919281721115, Actual momementum = 0.015394230373203754
Prediction = 0.05362824350595474, Actual momementum = -0.025680720806121826
Prediction = 0.027782868593931198, Actual momementum = 0.12492278963327408
Prediction = 0.05337393283843994, 

-0.18491950105139177